# Boundary Element Method

In [23]:
import numpy as np
import sympy as sp
import math as mt
import scipy.integrate as integrate
from tabulate import tabulate

<b>Boundaries</b>

In [41]:
def x11(t: float):
    return sp.cos(t) / 2

def x12(t: float):
    return sp.sin(t) / 2

def x21(t: float):
    return sp.cos(t)

def x22(t: float):
    return sp.sin(t)

<b>Derivatives</b>

In [3]:
t = sp.Symbol('t')
j = sp.Symbol('j')

In [4]:
def evalf_d(expr, tau: float, j: int):
    xj = expr.evalf(subs = {t: j * h})
    return xj + tau * (expr.evalf(subs = {t: (j + 1) * h}) - xj) 

In [5]:
def d_xj(x, tau: float, j: int):
    expr = sp.diff(x(t))
    return evalf_d(expr, tau, j)

In [6]:
def d2_xj(x, tau: float, j: int):
    expr = sp.diff(sp.diff(x(t)))
    return evalf_d(expr, tau, j)

In [7]:
def d_x1(t: float, j: int):
    return [d_xj(x11, t, j), d_xj(x12, t, j)]

def d_x2(t: float, j: int):
    return [d_xj(x21, t, j), d_xj(x22, t, j)]

<b>Integrals</b>

In [8]:
def quad(K):
    return integrate.quad(K, 0, 2 * np.pi)[0] / (2 * np.pi)

<b>Normals</b>

In [9]:
def v1(t: float, j: int):
    z = sp.sqrt(np.dot(x1(t, j), x1(t, j)))
    return [xj(x12, t, j) / z, - xj(x11, t, j) / z]

def v2(t: float, j: int):
    z = sp.sqrt(np.dot(x2(t, j), x2(t, j)))
    return [xj(x22, t, j) / z, - xj(x21, t, j) / z]

<b>Elements Count</b>

In [91]:
N = 8 # count
h = 2 * sp.pi / N # step

<b>Aproximation with Linear Function</b>

In [51]:
def xj(x, t: float, j: int):
    xj = x(j * h) + t * (x((j + 1) * h) - x(j * h))
    return xj

def x1(t: float, j: int):
    return [xj(x11, t, j), xj(x12, t, j)]

def x2(t: float, j: int):
    return [xj(x21, t, j), xj(x22, t, j)]

<b>Kernels</b>

In [96]:
def K11(t: float, tau: float, j: int, k: int):
    if t != tau:
        sub = np.subtract(x1(t, k), x1(tau, j))
        return ((xj(x11, t, k) - xj(x11, tau, j)) * d_xj(x12, tau, j) - (xj(x12, t, k) - xj(x12, tau, j)) * d_xj(x11, tau, j)) / mt.sqrt(np.dot(sub, sub))
    else:
        return (d_xj(x11, tau, j) * d2_xj(x12, tau, j) - d_xj(x12, tau, j) * d2_xj(x11, tau, j)) / np.dot(d_x1(tau, j), d_x1(tau, j))

def K22(t: float, tau: float, j: int, k: int):
    if t != tau:
        sub = np.subtract(x2(t, k), x2(tau, j))
        return ((xj(x21, t, k) - xj(x21, tau, j)) * d_xj(x22, tau, j) - (xj(x22, t, k) - xj(x22, tau, j)) * d_xj(x21, tau, j)) / mt.sqrt(np.dot(sub, sub))
    else:
        return - (d_xj(x21, tau, j) * d2_xj(x22, tau, j) - d_xj(x22, tau, j) * d2_xj(x21, tau, j)) / np.dot(d_x2(tau, j), d_x2(tau, j))

def K12(t: float, tau: float, j: int, k: int):
    sub = np.subtract(x1(t, k), x2(tau, j))
    return sp.log(1 / mt.sqrt(np.dot(sub, sub)))

def K21(t: float, tau: float, j: int, k: int):
    sub = np.subtract(x2(t, k), x1(tau, j))
    return np.dot(v2(t, k), v1(tau, j)) / np.dot(sub, sub) - 2 * np.dot(sub, v2(t, k)) * np.dot(sub, v1(tau, j)) / np.dot(sub,sub) ** 2

In [92]:
def matrix_A():
    matrix = np.zeros((2 * N, 2 * N))
    
    for k in range(0, N):
        
        print("Перший квадрат")
        for j in range(0, N):
            matrix[k][j] = quad(lambda tau: K11(np.pi, tau, j, k) * mt.sqrt(np.dot(x1(tau, j), x1(tau, j))))
            if k == j:
                matrix[k][j] += 1 / 2
        
        j = 0
        print("Другий  квадрат")
        for i in range(N, 2 * N):
            matrix[k][i] = quad(lambda tau: K12(np.pi, tau, j, k) * mt.sqrt(np.dot(x2(tau, j), x2(tau, j))))
            j += 1

    for k in range(N, 2 * N):
        
        print("Третій квадрат")
        for j in range(0, N):
            matrix[k][j] = quad(lambda tau: K21(np.pi, tau, j, k) * mt.sqrt(np.dot(x1(tau, j), x1(tau, j))))
            
        print("Четвертий квадрат")
        j = 0
        for j in range(N, 2 * N):
            matrix[k][j] = quad(lambda tau: K22(np.pi, tau, j, k) * mt.sqrt(np.dot(x2(tau, j), x2(tau, j))))
            if k == j:
                matrix[k][j] += 1 / 2 
            j += 1
            
    print(matrix)
    return matrix

In [14]:
def vector_b():
    vec_b = np.zeros((2 * N, 1))
    
    for i in range(0, N):
        vec_b[i][0] = 1
        
    for i in range(N, 2 * N):
        vec_b[i][0] = 0
        
    print(vec_b)
    return vec_b

In [69]:
def solve_system(A, b):
    m = np.linalg.solve(A, b)
    print(m)
    
    u = 0

    for i in range(0, int(len(m) / 2)):
        u += m[i][0]
        
    for i in range(int(len(m) / 2), len(m)):
        u += m[i][0]

    return u

In [97]:
A = matrix_A()

Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
[[-0.73865278 -1.06138484 -1.34387337 -1.60256546 -1.69930379 -1.61189332
  -1.39042159 -1.21926474 -1.19923111 -1.50068155 -2.37502693 -2.95983038
  -3.14616498 -3.02284365 -2.59471852 -1.88080912]
 [-1.21926474 -0.73865278 -1.06138484 -1.34387337 -1.60256546 -1.69930379
  -1.61189332 -1.39042159 -1.88080912 -1.19923111 -1.50068155 -2.37502693
  -2.95983038 -3.14616498 -3.02284365 -2.59471852]
 [-1.39042159 -1.21926474 -0.73865278 -1.06138484 -1.34387337 -1.60256546
  -1.69930379 -1

In [98]:
print(tabulate(A))
print("det = " + str(np.linalg.det(A)))

----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  --------  --------  --------  --------  --------  --------  --------  --------
-0.738653   -1.06138    -1.34387    -1.60257    -1.6993     -1.61189    -1.39042    -1.21926    -1.19923  -1.50068  -2.37503  -2.95983  -3.14616  -3.02284  -2.59472  -1.88081
-1.21926    -0.738653   -1.06138    -1.34387    -1.60257    -1.6993     -1.61189    -1.39042    -1.88081  -1.19923  -1.50068  -2.37503  -2.95983  -3.14616  -3.02284  -2.59472
-1.39042    -1.21926    -0.738653   -1.06138    -1.34387    -1.60257    -1.6993     -1.61189    -2.59472  -1.88081  -1.19923  -1.50068  -2.37503  -2.95983  -3.14616  -3.02284
-1.61189    -1.39042    -1.21926    -0.738653   -1.06138    -1.34387    -1.60257    -1.6993     -3.02284  -2.59472  -1.88081  -1.19923  -1.50068  -2.37503  -2.95983  -3.14616
-1.6993     -1.61189    -1.39042    -1.21926    -0.738653   -1.06138    -1.34387    -1.60257    -3.14616  -3.02284  -2.59472 

In [99]:
b = vector_b()
print(solve_system(A, b))

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[-0.09935938]
 [-0.09935938]
 [-0.09935938]
 [-0.09935938]
 [-0.09935938]
 [-0.09935938]
 [-0.09935938]
 [-0.09935938]
 [ 0.00320688]
 [ 0.00320688]
 [ 0.00320688]
 [ 0.00320688]
 [ 0.00320688]
 [ 0.00320688]
 [ 0.00320688]
 [ 0.00320688]]
-0.7692200103793105


In [48]:
for j in range(0, N):
    #res = quad(lambda tau: K11(np.pi, tau, j) * mt.sqrt(np.dot(x1(tau, j), x1(tau, j))))
    #print(res)
    print("j = " + str(j))
    print(K11(np.pi, 0, j))
    #print(mt.sqrt(np.dot(x1(0.4, j), x1(0.4, j))))
            

j = 0
-1.07079632679490
1.57079632679490
1/2
0
-1.07079632679490
1/2
1.57079632679490
0
-0.159154943091895
j = 1
-1.57079632679490
-1.07079632679490
0
1/2
-1.57079632679490
0
-1.07079632679490
1/2
-0.159154943091895
j = 2
1.07079632679490
-1.57079632679490
-1/2
0
1.07079632679490
-1/2
-1.57079632679490
0
-0.159154943091895
j = 3
1.57079632679490
1.07079632679490
0
-1/2
1.57079632679490
0
1.07079632679490
-1/2
-0.159154943091895
